In [17]:
import time
from numba import njit,prange,vectorize,int32,float64,int64,jit,cuda,float32
import numpy as np
import random
import tensorflow as tf

In [18]:
def preprocessing(tx_train, ty_train, tx_test, ty_test):
        
    py_train = ty_train.flatten()
    py_test = ty_test.flatten()
    
    px_test = tx_test.reshape(10000,input_size)
    px_train = tx_train.reshape(60000,input_size)
    
    px_train = np.asarray(px_train).astype(np.int32)
    py_train = np.asarray(py_train).astype(np.int32)
    px_test = np.asarray(px_test).astype(np.int32)
    py_test = np.asarray(py_test).astype(np.int32)

    return px_train, py_train, px_test, py_test   


In [19]:
class WiSARDrp:                                                     
    
    def __init__(self,input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number):
        self.input_size = input_size
        self.no_of_rand_pix_selec = no_of_rand_pix_selec
        self.nodes = nodes
        self.ram_address_count = ram_address_count
        self.dis_number = dis_number 


    def discriminator(self):
        discriminator = []
        accumulated_pos = []
        my_list = list(range(0,self.input_size))
        for i in range(self.dis_number):  #10
            ram = []
            random.shuffle(my_list)
            for j in range((int)((self.nodes))): #98    
                total_pos = []            
                positions = []
                positions = my_list[j*no_of_rand_pix_selec:j*no_of_rand_pix_selec+no_of_rand_pix_selec]
                accumulated_pos.append(positions)
                total_pos = np.vstack(positions)
                #print(total_pos)
                table = []
                dictionary = {}
                
                max = len("{0:b}".format(2**len(total_pos))) - 1
                for i in range(2**len(total_pos)):
                    x = (('0' * max) + "{0:b}".format(i))
                    x = x[len(x)-max:]
                    dictionary[x] = 0
                table.append(dictionary)
                #print(table)
                ram.append(table)
        
            di = []
            for j in range(len(ram)):
                for i in range(len(ram[j])):
                    for key, value in ram[j][i].items():
                        temp = [key,value]
                        di.append(temp)
                        
            discriminator.append(di)
    
        discriminator = np.asarray(discriminator).astype(np.int32)
        accumulated_pos = np.asarray(accumulated_pos).astype(np.int32)
        return discriminator, accumulated_pos
    
    
    @staticmethod
    @njit(parallel = True)
    def train_with_bleeching(d,pos,x_train, y_train):
        
        images = x_train
        lable = y_train    
        
        for i in range(len(images)):
            image = images[i]
            num = lable[i]
            all_ram_of_selected_discriminator = d[num]
            t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
            for i in range((int)(nodes)):
                part = all_ram_of_selected_discriminator[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                ratina_for_one_ram = t_ratina[i]
                
                #threshold = 0               
                n = []                                                                
                for ix in range(len(ratina_for_one_ram)):
                    pix = ratina_for_one_ram[ix]
                    if image[(pix-1)]>=1:
                        n.append(1)
                        #threshold = threshold + 1     
                    else:
                        n.append(0)
                
                num = 0
                for i in range(no_of_rand_pix_selec):
                    num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
            
                address_of_that_ram = (int)(num)
                for key in range(ram_address_count):
                    index = part[key]
                    #if threshold >= 1:          
                    if index[0] == address_of_that_ram:
                        index[1] += 1
                    else:
                        if index[1] != 0:
                            index[1] -= 1


In [20]:
input_size = 28*28
no_of_rand_pix_selec = 2**(3)     ## ** (must) no_of_rand_pix_selec = 2^(n) where n is 0,1,2...
nodes = int(input_size/no_of_rand_pix_selec)    #98
ram_address_count = 2**(no_of_rand_pix_selec)   #256
dis_number = 10                #10 i.e number of lables

In [21]:
(tx_train, ty_train), (tx_test, ty_test) = tf.keras.datasets.mnist.load_data()
#print(tx_train.shape)
px_train, py_train, px_test, py_test = preprocessing(tx_train, ty_train, tx_test, ty_test)
print(px_train.shape)
print(px_train[80].shape)
#print(px_train[80])

(60000, 784)
(784,)


In [22]:
w = WiSARDrp(input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number)
d, acc_pos = w.discriminator()
print(d.shape)

(10, 25088, 2)


In [23]:
starttrain = time.time()
w.train_with_bleeching(d,acc_pos,px_train[0:5000],py_train[0:5000])
endtrain = time.time()
print("time train = ",endtrain - starttrain)

time train =  1.8313839435577393


In [31]:
@cuda.jit
def add_kernel(d,pos,image,actual_lable,b,total_sum):
    tx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid
    
    start = tx + ty * block_size
    stride = block_size * grid_size
    
    
    #start = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value
    #stride = cuda.gridsize(1)
    ind_tot = 0
    for ix in range(start, dis_number, stride):
        
        t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
        sum_of_ram_output = 0
        dis = d[ix]
                
        for i in range(int(nodes)):
            part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
            ratina_for_one_ram = t_ratina[i]
                    
            #n = []  
            n = cuda.local.array(shape = 8, dtype = int32)
            index = 0
            for pix in ratina_for_one_ram:
                if image[(pix-1)]>=1:
                    n[index] = 1
                    #x = 0
                else:
                    n[index] = 0
                    #x = 0
                index += 1
            
            #print(index)
            num = 0
            for i in range(no_of_rand_pix_selec):
                num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
            address_of_that_ram = (int)(num)
                
            for key in range(len(part)):
                prt = part[key]
                if prt[0] == address_of_that_ram and prt[1]>=b:           #have to change this
                    sum_of_ram_output += 1
                
        total_sum[ind_tot] = sum_of_ram_output 
        #total_sum[ind_tot] = 11 + ind_tot
        ind_tot = ind_tot + 1
        
#@njit(parallel = True)
@jit
def test_with_bleaching(d,pos,x_test,y_test):
    right = 0
    wrong = 0
    images = x_test
    lable = y_test
    ct = 0.02
    b=1
        
    for i in range(len(images)):
        image = images[i]
        actual_lable = lable[i]
        
        threads_per_block = 1
        blocks_per_grid = 1
        
        #n = np.arange(8).astype(np.float32)
        n = dis_number
        x = np.arange(n).astype(np.float32)
        total_sum = np.empty_like(x)
        #total_sum=np.arange(10).astype(np.float32)
        #print(total_sum)
        
        
        #threads_per_block = 100
        #blocks_per_grid = 1
        #d_device = cuda.to_device(d)
        #pos_device = cuda.to_device(pos)
        #image_device = cuda.to_device(image)
        #actual_lable_device = cuda.to_device(actual_lable)
        #total_sum_device = cuda.device_array_like(total_sum)
        #add_kernel[blocks_per_grid, threads_per_block](d_device,pos_device,image_device,actual_lable_device,b,total_sum_device)   
        #print(total_sum)
        add_kernel[blocks_per_grid, threads_per_block](d,pos,image,actual_lable,b,total_sum)

        max_sum = 0
        sec_max = 0
        idx = 0
        
        #total_sum = total_sum_device.copy_to_host()
        
        for i in range(len(total_sum)):
            if max_sum < total_sum[i]:
                max_sum = total_sum[i]
                idx = i
                    
        for j in range(len(total_sum)):
            if sec_max < total_sum[j] and j!=idx:
                sec_max = total_sum[j]
                    
        index_of_dis = idx
        if index_of_dis == actual_lable:
            right += 1
        else:
            wrong += 1
        
        if max_sum == sec_max:
            confidence = 0
        else:
            confidence = 1 - float(sec_max)/float(max_sum)
            if confidence < ct:
                b += 1
        
    return right,wrong

In [32]:
starttest = time.time()
right,wrong = test_with_bleaching(d,acc_pos,px_test[0:100],py_test[0:100])
endtest = time.time()
print("time test = ",endtest - starttest)

time test =  2.337955951690674


In [33]:
print("number of right result = ",right)
print("number of wrong results = ",wrong)
accuracy = ((right)/(right+wrong))*100
print("accuracy by testing the model =",accuracy)
#    end = time.time()
#    print("total time = ",end - start)

number of right result =  55
number of wrong results =  45
accuracy by testing the model = 55.00000000000001


In [27]:
@njit(parallel = True)
def test_with_bleaching(d,pos,x_test,y_test):
    right = 0
    wrong = 0
    images = x_test
    lable = y_test
    ct = 0.02
    b=1
        
    for i in range(len(images)):
        image = images[i]
        actual_lable = lable[i]
        
        total_sum=[]
            
        for ix in range(dis_number):
                
            t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
            sum_of_ram_output = 0
            dis = d[ix]
                
            for i in range(int(nodes)):
                part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                ratina_for_one_ram = t_ratina[i]
                    
                n = []                                                                
                for pix in ratina_for_one_ram:
                    if image[(pix-1)]>=1:
                        n.append(1)
                    else:
                        n.append(0)
                    
                num = 0
                for i in range(no_of_rand_pix_selec):
                    num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
                address_of_that_ram = (int)(num)
                
                for key in range(len(part)):
                    prt = part[key]
                    if prt[0] == address_of_that_ram and prt[1]>=b:           #have to change this
                        sum_of_ram_output += 1
                
            total_sum.append(sum_of_ram_output)        
        
        #print(total_sum)
        max_sum = 0
        sec_max = 0
        idx = 0
            
        for i in range(len(total_sum)):
            if max_sum < total_sum[i]:
                max_sum = total_sum[i]
                idx = i
                    
        for j in range(len(total_sum)):
            if sec_max < total_sum[j] and j!=idx:
                sec_max = total_sum[j]
                    
        index_of_dis = idx
        if index_of_dis == actual_lable:
            right += 1
        else:
            wrong += 1
        
        if max_sum == sec_max:
            confidence = 0
        else:
            confidence = 1 - float(sec_max)/float(max_sum)
            if confidence < ct:
                b += 1
        
    return right,wrong